In [1]:
import script_automate.train_CNN

import tensorflow as tf
from tensorflow.python.keras.layers import Dense, InputLayer, Conv2D, MaxPool2D, Flatten, BatchNormalization
import os

from datetime import datetime
import numpy as np

In [2]:
import sys
print(os.getcwd())
print(sys.argv[2])
print(os.path.realpath('__file__'))
print(os.path.dirname(os.path.realpath('__file__')))

C:\Users\Muell\Documents\Programmieren\GitHub\neural-network-digital-counter-readout
C:\Users\Muell\AppData\Roaming\jupyter\runtime\kernel-d4814e2b-1b78-4cc5-a52c-51c782b8ae70.json
C:\Users\Muell\Documents\Programmieren\GitHub\neural-network-digital-counter-readout\__file__
C:\Users\Muell\Documents\Programmieren\GitHub\neural-network-digital-counter-readout


In [3]:
DefaultName = "dig"

Input_dir='ziffer_sortiert_resize'
Training_Percentage = 0.2

### Image Augmentation
Shift_Range = 1
Brightness_Range = 0.3
Rotation_Angle = 5
ZoomRange = 0.2

Batch_Size = 4

Epoch_Anz  = 10

In [4]:
DateNow = datetime.today().strftime('%Y%m%d')

model = script_automate.train_CNN.load_model(DefaultName, DateNow)

if model == None:
    model = tf.keras.Sequential()
    model.add(BatchNormalization(input_shape=(32,20,3)))
    model.add(Conv2D(32, (3, 3), padding='same', activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(512,activation="relu"))
    model.add(Dense(11, activation = "softmax"))

model.summary()
model.compile(loss= tf.keras.losses.categorical_crossentropy, 
              optimizer= tf.keras.optimizers.Adadelta(learning_rate=1.0, rho=0.95), 
              metrics = ["accuracy"])

Workfile does not exists - init model necessary
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 32, 20, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 20, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 10, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 10, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 5, 64)          36928     
_________________________________________________________________
max_pool

In [5]:
x_train, x_test, y_train, y_test, AnzahlBilder = script_automate.train_CNN.train_load_image(Input_dir, Training_Percentage)

0
(0,)
(0,)


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
train_iterator, validation_iterator = script_automate.train_CNN.get_iterator(x_train, x_test, y_train, y_test, 
                                                                             Shift_Range, 
                                                                             Brightness_Range, 
                                                                             Rotation_Angle, 
                                                                             ZoomRange, 
                                                                             Batch_Size)

In [ ]:
history = model.fit(train_iterator, 
                    validation_data = validation_iterator, 
                    epochs          = Epoch_Anz)

In [ ]:
## H5-Format
model.save('saved_model/' + DefaultName)
model.save('saved_model/' + DefaultName + "_" + DateNow)

In [ ]:
converter    = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open(DefaultName + ".tflite", "wb").write(tflite_model)


In [ ]:
from pathlib import Path
import tensorflow as tf

FileName = DefaultName + "_q.tflite"

def representative_dataset():
    for n in range(x_train[0].size):
      data = np.expand_dims(x_train[5], axis=0)
      yield [data.astype(np.float32)]
        
converter2 = tf.lite.TFLiteConverter.from_keras_model(model)
converter2.representative_dataset = representative_dataset
converter2.optimizations = [tf.lite.Optimize.DEFAULT]
converter2.representative_dataset = representative_dataset
tflite_quant_model = converter2.convert()

open(FileName, "wb").write(tflite_quant_model)
print(FileName)
Path(FileName).stat().st_size

In [ ]:
file_object = open('training_' + DefaultName + '.txt', 'a')
for x in range(np.size(history.history['loss'])):
    text = DateNow + "\t" + str(AnzahlBilder) + "\t"+ str(x+1) + "\t" + str(history.history['loss'][x]) + "\t" + str(history.history['val_loss'][x])
    print(text)
    file_object.write(text + "\n")
    
file_object.close()